# Classification thourgh deep learning 

(Classic feed forward with SGD for now, won't have the time to experiment on

RNN and LSTM to benefit from temporality of data. Going further section)

#### Import packages 

In [84]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
plt.style.use('seaborn-poster')

In [85]:
from sklearn.preprocessing import StandardScaler

#### Load prepared data set 

In [86]:
import data_formatter

data_set = pd.read_csv("prepared_dataset.csv")

#### Consider features only from the past event for now 

In [96]:
nb_events_as_features = 1

X, y = data_formatter.prepare_data_set(data_set, nb_events_as_features)

#### Train, validation, test split for simple algorithms comparison

Note that we can't use cross validation given the temporality of the data.

Cross validation would introduce *future bias* 

In [97]:
proportion_train = 0.8
proportion_validation = 0.1

cut_train = round(X.shape[0] * proportion_train)
cut_validation = round(X.shape[0] * (proportion_train + proportion_validation))

X_train, y_train = X.iloc[:cut_train,:], y.iloc[:cut_train]
X_validation, y_validation = X.iloc[cut_train:cut_validation,:], y.iloc[cut_train:cut_validation]
X_test, y_test = X.iloc[cut_validation:,:], y.iloc[cut_validation:]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

train_null_accuracy = y_train.value_counts()[0] / len(y_train)
test_null_accuracy = y_test.value_counts()[0] / len(y_test)

#### Convert up/down to one hot encoded output 

In [91]:
from keras.utils.np_utils import to_categorical

In [92]:
print(y_train[:3])
y_train = to_categorical(y_train)
print(y_train[:3])

0    0
1    1
2    1
Name: binaryOutput, dtype: int64
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]]


#### Import Keras components

In [93]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras import optimizers

#### Define and train NN

In [94]:
# Layer dimensions

N = X_train.shape[1]
H = 40
K = 2

# Stack layers and compile NN

model = Sequential()
model.add(Dense(H, input_dim=N))
model.add(Activation("tanh"))
model.add(Dense(K))
model.add(Activation("softmax"))

model.compile(optimizer=optimizers.SGD(lr=0.1),
              loss='categorical_crossentropy', metrics=['accuracy'])

print('{:15}: {:>12}'.format('Null accuracy', train_null_accuracy))

# Train NN

history = model.fit(X_train, y_train, epochs=15, batch_size=32)

Null accuracy  : 0.5040145157632117
Epoch 1/15
22045/22045 [==============================] - 0s - loss: 0.6859 - acc: 0.5655     
Epoch 2/15
22045/22045 [==============================] - 0s - loss: 0.6732 - acc: 0.5837     
Epoch 3/15
22045/22045 [==============================] - 0s - loss: 0.6707 - acc: 0.5873     
Epoch 4/15
22045/22045 [==============================] - 0s - loss: 0.6688 - acc: 0.5920     
Epoch 5/15
22045/22045 [==============================] - 0s - loss: 0.6673 - acc: 0.5932     
Epoch 6/15
22045/22045 [==============================] - 0s - loss: 0.6667 - acc: 0.5942     
Epoch 7/15
22045/22045 [==============================] - 0s - loss: 0.6660 - acc: 0.5965     
Epoch 8/15
22045/22045 [==============================] - 0s - loss: 0.6646 - acc: 0.5980     
Epoch 9/15
22045/22045 [==============================] - 0s - loss: 0.6649 - acc: 0.5970     
Epoch 10/15
22045/22045 [==============================] - 0s - loss: 0.6642 - acc: 0.5996     
Epoch 11/15
2

In [95]:
history.history

{'acc': [0.56552506233997479,
  0.58366976634908296,
  0.58725334542709895,
  0.59201633022144606,
  0.59324109775729661,
  0.59423905646727992,
  0.59650714447450193,
  0.59800408258003346,
  0.59700612380515961,
  0.59963710588726438,
  0.60285779088499003,
  0.60326604671990902,
  0.60226808801803711,
  0.60344749378033247,
  0.60421864367526357],
 'loss': [0.68586595794678162,
  0.67323588640896959,
  0.67071030536296006,
  0.66883709198710972,
  0.66730123401418184,
  0.66668719395836296,
  0.66595028315739013,
  0.66459086720812044,
  0.66489869684122893,
  0.66422362694207115,
  0.6626843394187335,
  0.66261714465441302,
  0.66256572368004929,
  0.66128203154259135,
  0.66150275946041981]}

#### Evaluate the NN on out of sample data

In [98]:
y_test = to_categorical(y_test)

In [99]:
evaluation = model.evaluate(X_test, y_test, batch_size=32, verbose=1)

2688/2756 [============================>.] - ETA: 0s

In [100]:
evaluation

[0.75167188173800659, 0.52140783744557329]